In [3]:
file_name = "VSD"
class_name = "aorta"

In [4]:
import os
import SimpleITK as sitk

# Generate MHD sigmoid mask

indir = "./segmentations/" + file_name + "/";
outdir = "./segmentations/" + file_name + "/mhd/";

print(indir)
print(outdir)

if not os.path.exists(outdir) :
    os.makedirs(outdir)
    
for file in os.listdir(indir):
    if os.path.isfile(indir + file) and ".nii.gz" in file :
        img = sitk.ReadImage(indir + file)
        
        path = outdir + file.replace(".nii.gz", ".mhd")
        scale_filter = sitk.ShiftScaleImageFilter()
        scale_filter.SetScale(100)
        res = scale_filter.Execute(img)

        minmax_filter = sitk.MinimumMaximumImageFilter()
        minmax_filter.Execute(img)
        
        if minmax_filter.GetMaximum() != 0 :
            sitk.WriteImage(res, path)

./segmentations/VSD/
./segmentations/VSD/mhd/


Mesh generation from MHD voxel files using FAST extraction

In [ ]:
from stl import mesh as stmesh
import math
import numpy as np
import fast

indir = "./segmentations/" + file_name + "/mhd/";
outdir = "./meshes/stl";

# Spine
#files = [\
#    "vertebrae_L1",\
#    "vertebrae_L2",\
#    "vertebrae_L3",\
#    "vertebrae_L4",\
#    "vertebrae_L5",\
#    "vertebrae_T1",\
#    "vertebrae_T10",\
#    "vertebrae_T11",\
#    "vertebrae_T12",\
#    "vertebrae_T2",\
#    "vertebrae_T3",\
#    "vertebrae_T4",\
#    "vertebrae_T5",\
#    "vertebrae_T6",\
#    "vertebrae_T7",\
#    "vertebrae_T8",\
#    "vertebrae_T9",\
#    "sacrum"];

# Heart
#files = [\
#    "heart_atrium_left",\
#    "heart_atrium_right",\
#    "heart_myocardium",\
#    "heart_ventricle_left",\
#    "heart_ventricle_right"];

# Aorta
files = ["aorta"]

verts = []
faces = []

offset = 0

for file in files:
    importer = fast.ImageFileImporter.create(indir + file + ".mhd")
    smoothing = fast.GaussianSmoothing.create(stdDev=2.0).connect(importer)
    extraction = fast.SurfaceExtraction.create(threshold=50).connect(smoothing)
    mesh = extraction.runAndGetOutputData()
    access = mesh.getMeshAccess(fast.ACCESS_READ)

    for vertex in access.getVertices():
        verts.append(vertex.getPosition().reshape(1, 3))

    for face in access.getTriangles():
        faces.append([offset + face.getEndpoint1(), offset + face.getEndpoint2(), offset + face.getEndpoint3()])
        
    offset = len(verts)
    
    print(file)
    
verts = np.array(verts)
faces = np.array(faces)
    
model = stmesh.Mesh(np.zeros(faces.shape[0], dtype=stmesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        model.vectors[i][j] = verts[f[j],:]
    
model.save(outdir + "output.stl")

In [ ]:
from vmtk import pypes
from vmtk import vmtkscripts
import pipes

args = "vmtkimagereader -ifile ./segmentations/" + file_name + "/mhd/" + class_name + ".mhd --pipe vmtkimagewriter -ofile ./segmentations/" + file_name + "/vti/" + class_name + ".vti"
                                                                
res = pypes.PypeRun(args)

In [ ]:
# Segmenting the aorta via levelsets

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkimagereader -ifile ./segmentations/" + file_name + "/mhd/" + class_name + ".mhd --pipe vmtklevelsetsegmentation -ofile ./meshes/vtp/" + class_name + ".vtk"

pypes.PypeRun(args)

In [ ]:
# Clipping mesh inlets and outlets
  
from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfaceclipper -ifile ./meshes/vtp/" + class_name + ".vtk -ofile ./meshes/vtp/" + class_name + "_clip.vtk"

res = pypes.PypeRun(args)

In [ ]:
# Surface smoothing

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacesmoothing -ifile ./meshes/vtp/" + class_name + "_clip.vtk -passband 0.1 -iterations 30 -ofile ./meshes/vtp/" + class_name + "_smooth.vtk"

res = pypes.PypeRun(args)

In [ ]:
# Surface subdivision

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacesubdivision -ifile ./meshes/vtp/" + class_name + "_smooth.vtk -ofile ./meshes/vtp/" + class_name + "_subdiv.vtk -method butterfly"

res = pypes.PypeRun(args)

In [ ]:
# Comparison

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacereader -ifile ./meshes/vtp/" + class_name + "_clip.vtk --pipe vmtksurfacesmoothing -iterations 30 -passband 0.1 --pipe vmtkrenderer --pipe vmtksurfaceviewer -display 0 --pipe vmtksurfaceviewer -i @vmtksurfacereader.o -color 1 0 0 -display 1"

res = pypes.PypeRun(args)

In [ ]:
# Centerline computation

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkcenterlines -ifile ./meshes/vtp/" + class_name + "_smooth.vtk -ofile ./meshes/vtp/" + class_name + "_centerline.vtk"

res = pypes.PypeRun(args)

In [ ]:
# Centerline computation visualization

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacereader -ifile ./meshes/vtp/" + class_name + "_smooth.vtk --pipe vmtkcenterlines --pipe vmtkrenderer --pipe vmtksurfaceviewer -opacity 0.25 --pipe vmtksurfaceviewer -i @vmtkcenterlines.voronoidiagram -array MaximumInscribedSphereRadius --pipe vmtksurfaceviewer -i @vmtkcenterlines.o"

res = pypes.PypeRun(args)

Centerline extraction using skeletonization with voxel structures

In [ ]:
# Alternative voxel-based centerline extraction

import SimpleITK as sitk
import os
import numpy as np
import time
from scipy.ndimage import binary_erosion

file = "./segmentations/" + file_name + "/mhd/" + class_name ".mhd"

img = sitk.ReadImage(file)

array = sitk.GetArrayFromImage(img)

eroded = array

start = time.perf_counter()

print(np.max(array))

for i in range(0, 10):
    eroded = binary_erosion(eroded)
    array += eroded
    
end = time.perf_counter()

maxval = np.max(array)

print(end - start)
print(maxval)

centerline = []

size = array.shape

for x in range(0, size[0]):
    for y in range(0, size[1]):
        for z in range(0, size[2]):
            if (array[x][y][z] == maxval):
                centerline.append((x, y, z))

Mesh extraction using GMSH from prepared VTK surface mesh

In [ ]:
import os
import vtk

outdir = "./meshes/";

# Aorta
filepath = "./meshes/vtp/" + class_name + "_smooth.vtk"

verts = []
faces = []

offset = 0

basename = os.path.basename(filepath)
print("Copying file:", basename)
basename = os.path.splitext(basename)[0]
outfile = os.path.join(outdir, basename+".stl")
reader = vtk.vtkGenericDataObjectReader()
reader.SetFileName(filepath)
reader.Update()
writer = vtk.vtkSTLWriter()
writer.SetInputConnection(reader.GetOutputPort())
writer.SetFileName(outfile)
writer.Write()

In [4]:
import gmsh
import sys
import os
import math

gmsh.initialize(sys.argv)

# merge STL, create surface patches that are reparametrizable (so we can remesh
# them) and compute the parametrizations
gmsh.merge(os.path.join('./meshes/vtp/aorta.stl'))
gmsh.model.mesh.classifySurfaces(math.pi, True, True)
gmsh.model.mesh.createGeometry()

# make extrusions only return "top" surfaces and volumes, not lateral surfaces
gmsh.option.setNumber('Geometry.ExtrudeReturnLateralEntities', 0)

# extrude a boundary layer of 4 elements using mesh normals (thickness = 0.5)
gmsh.model.geo.extrudeBoundaryLayer(gmsh.model.getEntities(2), [4], [1.0], True)

# extrude a second boundary layer in the opposite direction (note the `second ==
# True' argument to distinguish it from the first one)
e = gmsh.model.geo.extrudeBoundaryLayer(gmsh.model.getEntities(2), [4], [-1.0],
                                        True, True)

# get "top" surfaces created by extrusion
top_ent = [s for s in e if s[0] == 2]
top_surf = [s[1] for s in top_ent]

# get boundary of top surfaces, i.e. boundaries of holes
gmsh.model.geo.synchronize()
bnd_ent = gmsh.model.getBoundary(top_ent)
bnd_curv = [c[1] for c in bnd_ent]

# create plane surfaces filling the holes
loops = gmsh.model.geo.addCurveLoops(bnd_curv)
for l in loops:
    top_surf.append(gmsh.model.geo.addPlaneSurface([l]))

# create the inner volume
gmsh.model.geo.addVolume([gmsh.model.geo.addSurfaceLoop(top_surf)])
gmsh.model.geo.synchronize()

# use MeshAdapt for the resulting not-so-smooth parametrizations
gmsh.option.setNumber('Mesh.Algorithm', 1)
gmsh.option.setNumber('Mesh.MeshSizeFactor', 0.1)

if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()
